In [245]:
import scipy 
import numpy as np

def em_single(priors,observations):
    counts = {'A': {'H': 0, 'T': 0}, 'B': {'H': 0, 'T': 0}}
    # priors = [0.6,0.5]
    theta_A = priors[0,0]
    theta_B = priors[1,0]
    for observation in observations:
        len_observation = observation.sum()
        num_heads = observation[0]
        num_tails = observation[1]
    #二项分布求解公式
        contribution_A = scipy.stats.binom.pmf(num_heads,len_observation,theta_A)
        contribution_B = scipy.stats.binom.pmf(num_heads,len_observation,theta_B)
        weight_A = contribution_A / (contribution_A + contribution_B)
        weight_B = contribution_B / (contribution_A + contribution_B)
        counts['A']['H'] += weight_A*num_heads
        counts['A']['T'] += weight_A*num_tails
        counts['B']['H'] += weight_B*num_heads
        counts['B']['T'] += weight_B*num_tails
#M step
    new_theta_A = counts['A']['H']/(counts['A']['T']+counts['A']['H'])
    new_theta_B = counts['B']['H']/(counts['B']['T']+counts['B']['H'])
    return np.array([[new_theta_A,1-new_theta_A],
            [new_theta_B,1-new_theta_B]])
    
def em_test(observations,prior,tol = 1e-3,iterations=10000):
    iteration = 0;
    while iteration < iterations:
        new_prior = em_single(prior,observations)
        if np.abs(prior[0,0]-new_prior[0,0]) < tol:
            break
        else:
            prior = new_prior
            iteration +=1
        # print(f'第{iteration}次迭代，参数如下:\n{prior}')
        print("Iteration: %d" % (iteration))
        print("theta_B = %.2f, theta_C = %.2f" 
              % (prior[0,0], prior[1,0]))
    return prior

In [246]:
observed_data = np.array([(5,5), (8,2), (8,2), (4,6), (7,3)])
thetas = np.array([[0.6, 0.4], [0.5, 0.5]])
thetas = em_test(observed_data,thetas)
print(thetas)

Iteration: 1
theta_B = 0.68, theta_C = 0.58
Iteration: 2
theta_B = 0.69, theta_C = 0.58
Iteration: 3
theta_B = 0.70, theta_C = 0.58
Iteration: 4
theta_B = 0.70, theta_C = 0.58
Iteration: 5
theta_B = 0.71, theta_C = 0.57
Iteration: 6
theta_B = 0.71, theta_C = 0.57
Iteration: 7
theta_B = 0.71, theta_C = 0.57
Iteration: 8
theta_B = 0.71, theta_C = 0.56
Iteration: 9
theta_B = 0.72, theta_C = 0.56
Iteration: 10
theta_B = 0.72, theta_C = 0.56
Iteration: 11
theta_B = 0.72, theta_C = 0.56
[[0.71851872 0.28148128]
 [0.5596601  0.4403399 ]]


In [248]:

# EM
# 导入numpy库 
import numpy as np

### EM算法过程函数定义
def em(data, thetas, max_iter=30, eps=1e-6):
    '''
    输入：
    data：观测数据
    thetas：初始化的估计参数值
    max_iter：最大迭代次数
    eps：收敛阈值
    输出：
    thetas：估计参数
    '''
    # 初始化似然函数值
    ll_old = -np.infty
    for i in range(max_iter):
        ### E步：求隐变量分布
        # 对数似然
        log_like = np.array([np.sum(data * np.log(theta), axis=1) for theta in thetas]) # 单个,（5，2）数组[5*ln0.6,5*ln0.4] 按列相加 -> （5，1）数组 [5*ln0.6+5*ln0.4]   
        # 似然
        like = np.exp(log_like)  #e^(5*ln0.6+5*ln0.4) 等价于计算0.6^5 * 0.4^5 
        # (2,5)数组  
        #like.sum(axis = 0) （2，5）数组 按行相加-> (1,5)数组 
        # 求隐变量分布
        ws = like/like.sum(axis = 0)# (2，5）数组  
        # 概率加权
        vs = np.array([w[:, None] * data for w in ws]) #计算期望  （2，5，2）数组
        ### M步：更新参数值
        thetas = np.array([v.sum(axis = 0)/v.sum() for v in vs]) #v.sum(axis = 0) （5，2）数组变成 （1，2）数组 -> (2,1,2) [[thetasA1,thetasA2],[thetasB1,thetasB1]]
        # 更新似然函数
        ll_new = np.sum(ws * log_like)
        print("Iteration: %d" % (i+1))
        print("theta_B = %.2f, theta_C = %.2f, ll = %.2f" 
              % (thetas[0,0], thetas[1,0], ll_new))
        # 满足迭代条件即退出迭代
        if np.abs(ll_new - ll_old) < eps: 
            break
        ll_old = ll_new
    return thetas

In [249]:
# 观测数据，5次独立试验，每次试验10次抛掷的正反次数
# 比如第一次试验为5次正面5次反面
observed_data = np.array([(5,5), (8,2), (8,2), (4,6), (7,3)])
# 初始化参数值，即硬币B的正面概率为0.6，硬币C的正面概率为0.5
thetas = np.array([[0.6, 0.4], [0.5, 0.5]])
thetas = em(observed_data, thetas, max_iter=30, eps=1e-3)
print(thetas)

Iteration: 1
theta_B = 0.68, theta_C = 0.58, ll = -33.09
Iteration: 2
theta_B = 0.69, theta_C = 0.58, ll = -32.34
Iteration: 3
theta_B = 0.70, theta_C = 0.58, ll = -32.28
Iteration: 4
theta_B = 0.70, theta_C = 0.58, ll = -32.23
Iteration: 5
theta_B = 0.71, theta_C = 0.57, ll = -32.18
Iteration: 6
theta_B = 0.71, theta_C = 0.57, ll = -32.13
Iteration: 7
theta_B = 0.71, theta_C = 0.57, ll = -32.09
Iteration: 8
theta_B = 0.71, theta_C = 0.56, ll = -32.06
Iteration: 9
theta_B = 0.72, theta_C = 0.56, ll = -32.03
Iteration: 10
theta_B = 0.72, theta_C = 0.56, ll = -32.00
Iteration: 11
theta_B = 0.72, theta_C = 0.56, ll = -31.98
Iteration: 12
theta_B = 0.72, theta_C = 0.56, ll = -31.97
Iteration: 13
theta_B = 0.72, theta_C = 0.56, ll = -31.95
Iteration: 14
theta_B = 0.72, theta_C = 0.56, ll = -31.94
Iteration: 15
theta_B = 0.72, theta_C = 0.56, ll = -31.94
Iteration: 16
theta_B = 0.72, theta_C = 0.56, ll = -31.93
Iteration: 17
theta_B = 0.72, theta_C = 0.56, ll = -31.93
Iteration: 18
theta_B =